# 2. Working with data

In [ ]:
import torch

## A simple example

### Dataset

In [ ]:
from torch.utils.data import Dataset

In [ ]:
class DummyDataset(Dataset):
    def __init__(self):
        self.data = torch.rand(10)
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        sample = self.data[index]
        label = sample > 0.5
        return (sample, label)

In [ ]:
dataset = DummyDataset()

In [ ]:
dataset.data

In [ ]:
dataset[0]  # It returns a tuple (data example, class label)

### Dataloader

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
loader = DataLoader(dataset, batch_size=5, shuffle=True)

In [ ]:
for sample, label in loader:
    print(sample, label)
    break

## A real example

### Alien vs Predator Dataset 

In [ ]:
!tree -nd ../alien-vs-predator

![predator](../alien-vs-predator/train/predator/10.jpg)
![alien](../alien-vs-predator/train/alien/10.jpg)

In [ ]:
from pathlib import Path
from PIL import Image


class AlienPredatorDataset(Dataset):
    def __init__(self, root, split):
        self.root = root
        self.split = split
        
        # Load and save all image paths
        self.img_instances = []
        
        for img_path in Path(root, split, "alien").glob("*.jpg"):
            self.img_instances.append((img_path, 0))
            
        for img_path in Path(root, split, "predator").glob("*.jpg"):
            self.img_instances.append((img_path, 1))
    
    
    def __len__(self):
        return len(self.img_instances)
    
    
    def __getitem__(self, index):
        path, target = self.img_instances[index]
        
        with open(path, 'rb') as f:
            img = Image.open(f).convert('RGB')
            
        return (img, target)

In [ ]:
dataset = AlienPredatorDataset("../alien-vs-predator/", "train")

In [ ]:
len(dataset)

In [ ]:
dataset[0]  # Here again it returns a tuple (image, class label)

In [ ]:
dataset[1]

In [ ]:
dataset[0][0]

Use transforms to get tensors of similar size

### Torchvision's transforms

The torchvision package consists of popular datasets, model architectures, and common image transformations for computer vision.

In [ ]:
from torchvision.transforms import ToTensor, ToPILImage, RandomCrop

In [ ]:
crop_transform = RandomCrop(100)

In [ ]:
img = dataset[0][0]
img

In [ ]:
crop_transform(img)

In [ ]:
from torchvision.transforms import Compose

In [ ]:
all_transforms = Compose((
    RandomCrop(100),
    ToTensor(),
))

In [ ]:
all_transforms(img)

In [ ]:
all_transforms(img).shape

Let's apply it to our AlienPredatorDataset !

### Dataloader

In [ ]:
loader = DataLoader(dataset, batch_size=5, shuffle=True) # workers

for sample, label in loader:
    print(sample.shape, label)
    break

## Using torchvision ImageFolder

In [ ]:
from torchvision.datasets import ImageFolder

In [ ]:
dataset = ImageFolder(root="../alien-vs-predator/", transform=all_transforms)

In [ ]:
dataset[0]

---
# Building our training loop (2 / 5)

In [ ]:
# INITIALIZATION

import torch
from torch.utils.data import DataLoader
from torchvision.transforms import Compose, ToTensor, RandomCrop
from torchvision.datasets import ImageFolder

device = torch.device("cpu")

transform = Compose((RandomCrop((50, 50)), ToTensor()))
dataset = ImageFolder(root="../alien-vs-predator/", transform=transform)
loader = DataLoader(dataset, batch_size=5, shuffle=True, num_workers=5)

In [ ]:
# TRAINING LOOP

for samples, labels in loader:
    samples = samples.to(device)
    labels = labels.to(device)
    # compute predictions with model
    # compute the loss
    # compute gradients
    # update model parameters